In [250]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
from scipy import stats

In [251]:
df = pd.read_csv('home_price_original.csv') 
print(df.isnull().sum())
rows_with_nulls = df.isnull().any(axis=1).sum()
print(f"Number of rows with null values: {rows_with_nulls}")

Net_Metrekare             0
Brüt_Metrekare            0
Oda_Sayısı              260
Bulunduğu_Kat          2297
Eşya_Durumu            7398
Binanın_Yaşı              0
Isıtma_Tipi               0
Fiyat                     0
Şehir                     0
Binanın_Kat_Sayısı        0
Kullanım_Durumu           0
Yatırıma_Uygunluk      5576
Takas                  5895
Tapu_Durumu           10566
Banyo_Sayısı             46
dtype: int64
Number of rows with null values: 15135


The removal of extreme and impossible outliers 

In [252]:
df = pd.read_csv('home_price_original.csv')
df.isnull().sum()
#df.dropna(inplace=True)
#output_file = "home_price_cleaned.csv"
#df.to_csv(output_file, index=False)
print(df.isnull().sum())

Net_Metrekare             0
Brüt_Metrekare            0
Oda_Sayısı              260
Bulunduğu_Kat          2297
Eşya_Durumu            7398
Binanın_Yaşı              0
Isıtma_Tipi               0
Fiyat                     0
Şehir                     0
Binanın_Kat_Sayısı        0
Kullanım_Durumu           0
Yatırıma_Uygunluk      5576
Takas                  5895
Tapu_Durumu           10566
Banyo_Sayısı             46
dtype: int64


Fix for the easy to do null values oda, esya, yatırıma, takas, banyo 

In [253]:
# fix for oda, esya, yatırıma, takas, banyo nulls

# Read the CSV file
df = pd.read_csv('home_price_original.csv')

print("Original data info:")
null_summary = df.isnull().sum()
print(null_summary)
print(f"\nData types:\n{df.dtypes}")

# Show rows that contain any null values
null_rows = df[df.isnull().any(axis=1)]
print(f"\nRows with null values: {len(null_rows)}")
print(f"Original shape: {df.shape}")

# Check for duplicate rows
duplicate_count = df.duplicated().sum()
print(f"Number of duplicate rows: {duplicate_count}")

# Remove duplicates 
if duplicate_count > 0:
    df = df.drop_duplicates()
    print("Duplicates removed.")
else:
    print("No duplicates found.")

print(f"Shape after removing duplicates: {df.shape}")

# FIX NULL VALUES ACCORDING TO SPECIFICATIONS

print("\n" + "="*50)
print("FIXING NULL VALUES")
print("="*50)

# Store original dtypes for reference
original_dtypes = df.dtypes

# 1. For Oda_Sayısı null values, replace with Banyo_Sayısı + 1
print("\n1. Fixing Oda_Sayısı (Room Count)...")
print(f"   Null count before: {df['Oda_Sayısı'].isnull().sum()}")
print(f"   Data type: {df['Oda_Sayısı'].dtype}")

# Fill Oda_Sayısı nulls with Banyo_Sayısı + 1
mask_oda_null = df['Oda_Sayısı'].isnull()
# Ensure we're working with float values
df.loc[mask_oda_null, 'Oda_Sayısı'] = df.loc[mask_oda_null, 'Banyo_Sayısı'] + 1.0

print(f"   Null count after: {df['Oda_Sayısı'].isnull().sum()}")

# 2. For Eşya_Durumu null values, set to "Boş"
print("\n2. Fixing Eşya_Durumu (Furniture Status)...")
print(f"   Null count before: {df['Eşya_Durumu'].isnull().sum()}")
print(f"   Data type: {df['Eşya_Durumu'].dtype}")

df['Eşya_Durumu'] = df['Eşya_Durumu'].fillna('Boş')
print(f"   Null count after: {df['Eşya_Durumu'].isnull().sum()}")

# 3. For Bulunduğu_Kat (Floor number) - We'll leave as is since no instruction was given
print(f"\n3. Bulunduğu_Kat (Floor) null values remain: {df['Bulunduğu_Kat'].isnull().sum()}")
print(f"   Data type: {df['Bulunduğu_Kat'].dtype}")

# 4. For Yatırıma_Uygunluk null values, set to "bilinmiyor"
print("\n4. Fixing Yatırıma_Uygunluk (Investment Suitability)...")
print(f"   Null count before: {df['Yatırıma_Uygunluk'].isnull().sum()}")
print(f"   Data type: {df['Yatırıma_Uygunluk'].dtype}")

df['Yatırıma_Uygunluk'] = df['Yatırıma_Uygunluk'].fillna('Bilinmiyor')
print(f"   Null count after: {df['Yatırıma_Uygunluk'].isnull().sum()}")

# 5. For Takas null values, set to "yok"
print("\n5. Fixing Takas (Exchange)...")
print(f"   Null count before: {df['Takas'].isnull().sum()}")
print(f"   Data type: {df['Takas'].dtype}")

df['Takas'] = df['Takas'].fillna('Yok')
print(f"   Null count after: {df['Takas'].isnull().sum()}")

# 6. For Tapu_Durumu (Deed Status) - We'll leave as is since no instruction was given
print(f"\n6. Tapu_Durumu (Deed Status) null values remain: {df['Tapu_Durumu'].isnull().sum()}")
print(f"   Data type: {df['Tapu_Durumu'].dtype}")

# 7. For Banyo_Sayısı null values, set to Oda_Sayısı - 1
print("\n7. Fixing Banyo_Sayısı (Bathroom Count)...")
print(f"   Null count before: {df['Banyo_Sayısı'].isnull().sum()}")
print(f"   Data type: {df['Banyo_Sayısı'].dtype}")

# Fill Banyo_Sayısı nulls with Oda_Sayısı - 1
mask_bath_null = df['Banyo_Sayısı'].isnull()
df.loc[mask_bath_null, 'Banyo_Sayısı'] = df.loc[mask_bath_null, 'Oda_Sayısı'] - 1.0
print(f"   Null count after: {df['Banyo_Sayısı'].isnull().sum()}")

# Handle circular dependency issues
print("\n" + "="*50)
print("HANDLING REMAINING NULL VALUES")
print("="*50)

# Check for any remaining Oda_Sayısı nulls (in case Banyo_Sayısı was also null)
remaining_oda_nulls = df['Oda_Sayısı'].isnull().sum()
if remaining_oda_nulls > 0:
    print(f"\nWarning: {remaining_oda_nulls} Oda_Sayısı values are still null.")
    print("These occur when both Oda_Sayısı and Banyo_Sayısı were null.")
    print("Setting remaining Oda_Sayısı nulls to median value...")
    
    # Calculate median of non-null values
    median_oda = df['Oda_Sayısı'].median()
    print(f"Median Oda_Sayısı: {median_oda}")
    
    df['Oda_Sayısı'] = df['Oda_Sayısı'].fillna(median_oda)
    
    # Now recalculate Banyo_Sayısı for rows where it's still null
    mask_bath_still_null = df['Banyo_Sayısı'].isnull()
    df.loc[mask_bath_still_null, 'Banyo_Sayısı'] = df.loc[mask_bath_still_null, 'Oda_Sayısı'] - 1.0

# Check for any remaining Banyo_Sayısı nulls
remaining_bath_nulls = df['Banyo_Sayısı'].isnull().sum()
if remaining_bath_nulls > 0:
    print(f"\nWarning: {remaining_bath_nulls} Banyo_Sayısı values are still null.")
    print("Setting remaining Banyo_Sayısı nulls to median value...")
    
    median_bath = df['Banyo_Sayısı'].median()
    print(f"Median Banyo_Sayısı: {median_bath}")
    
    df['Banyo_Sayısı'] = df['Banyo_Sayısı'].fillna(median_bath)

# Final check for all nulls
print("\n" + "="*50)
print("FINAL NULL VALUE CHECK")
print("="*50)

null_counts_after = df.isnull().sum()
print("\nNull values after cleaning:")
for col in df.columns:
    count = null_counts_after[col]
    if count > 0:
        print(f"  {col}: {count} nulls (dtype: {df[col].dtype})")

rows_with_nulls = df.isnull().any(axis=1).sum()
print(f"\nRows with any null values after cleaning: {rows_with_nulls}")

print(f"\nData types after cleaning:")
print(df.dtypes)

# Save the cleaned dataframe back to a new file
cleaned_filename = 'home_price_cleaned.csv'
df.to_csv(cleaned_filename, index=False)
print(f"\n" + "="*50)
print(f"Cleaned data saved to '{cleaned_filename}'")
print(f"Final shape: {df.shape}")
print("="*50)

# Optional: Create a detailed cleaning report
try:
    with open('cleaning_report.txt', 'w') as f:
        f.write("DATA CLEANING REPORT\n")
        f.write("=" * 60 + "\n\n")
        
        f.write("FILE INFORMATION\n")
        f.write("-" * 40 + "\n")
        f.write(f"Original file: home_price_original.csv\n")
        f.write(f"Cleaned file: {cleaned_filename}\n")
        f.write(f"Original rows: 20131\n")
        f.write(f"Final rows: {len(df)}\n")
        f.write(f"Duplicates removed: {duplicate_count}\n\n")
        
        f.write("NULL VALUE SUMMARY - BEFORE CLEANING\n")
        f.write("-" * 40 + "\n")
        for col, count in null_summary.items():
            if count > 0:
                f.write(f"{col:25} {count:6} nulls ({df[col].dtype})\n")
        f.write(f"\nTotal rows with nulls: {len(null_rows)}\n\n")
        
        f.write("CLEANING ACTIONS PERFORMED\n")
        f.write("-" * 40 + "\n")
        f.write("1. Oda_Sayısı: null → Banyo_Sayısı + 1\n")
        f.write("2. Eşya_Durumu: null → 'Boş'\n")
        f.write("3. Yatırıma_Uygunluk: null → 'bilinmiyor'\n")
        f.write("4. Takas: null → 'yok'\n")
        f.write("5. Banyo_Sayısı: null → Oda_Sayısı - 1\n")
        f.write("6. Remaining Oda_Sayısı nulls: filled with median\n")
        f.write("7. Remaining Banyo_Sayısı nulls: filled with median\n\n")
        
        f.write("NULL VALUE SUMMARY - AFTER CLEANING\n")
        f.write("-" * 40 + "\n")
        for col in df.columns:
            count = df[col].isnull().sum()
            if count > 0:
                f.write(f"{col:25} {count:6} nulls ({df[col].dtype})\n")
        f.write(f"\nTotal rows with nulls: {rows_with_nulls}\n")
        
        f.write("\nDATA TYPE SUMMARY\n")
        f.write("-" * 40 + "\n")
        for col in df.columns:
            f.write(f"{col:25} {str(df[col].dtype):10}\n")
    
    print("Detailed cleaning report saved to 'cleaning_report.txt'")
except Exception as e:
    print(f"Note: Could not save cleaning report: {e}")

# Show sample of cleaned data
print("\n" + "="*50)
print("SAMPLE OF CLEANED DATA (first 5 rows)")
print("="*50)
print(df.head())

# Show the columns that still have null values
remaining_null_cols = [col for col in df.columns if df[col].isnull().sum() > 0]
if remaining_null_cols:
    print(f"\nNote: These columns still have null values: {remaining_null_cols}")
    print("You may want to handle these separately.")
else:
    print("\nAll null values have been successfully handled!")

Original data info:
Net_Metrekare             0
Brüt_Metrekare            0
Oda_Sayısı              260
Bulunduğu_Kat          2297
Eşya_Durumu            7398
Binanın_Yaşı              0
Isıtma_Tipi               0
Fiyat                     0
Şehir                     0
Binanın_Kat_Sayısı        0
Kullanım_Durumu           0
Yatırıma_Uygunluk      5576
Takas                  5895
Tapu_Durumu           10566
Banyo_Sayısı             46
dtype: int64

Data types:
Net_Metrekare           int64
Brüt_Metrekare        float64
Oda_Sayısı            float64
Bulunduğu_Kat          object
Eşya_Durumu            object
Binanın_Yaşı           object
Isıtma_Tipi            object
Fiyat                 float64
Şehir                  object
Binanın_Kat_Sayısı      int64
Kullanım_Durumu        object
Yatırıma_Uygunluk      object
Takas                  object
Tapu_Durumu            object
Banyo_Sayısı          float64
dtype: object

Rows with null values: 15135
Original shape: (20326, 15)
Number of du

Null values left are Bulunduğu_Kat, Tapu_Durumu

In [254]:
df =pd.read_csv('home_price_cleaned.csv')
print(df.isnull().sum())
rows_with_nulls = df.isnull().any(axis=1).sum()
print(f"Number of rows with null values: {rows_with_nulls}")

Net_Metrekare             0
Brüt_Metrekare            0
Oda_Sayısı                0
Bulunduğu_Kat          2272
Eşya_Durumu               0
Binanın_Yaşı              0
Isıtma_Tipi               0
Fiyat                     0
Şehir                     0
Binanın_Kat_Sayısı        0
Kullanım_Durumu           0
Yatırıma_Uygunluk         0
Takas                     0
Tapu_Durumu           10472
Banyo_Sayısı              0
dtype: int64
Number of rows with null values: 11587


the Avg price of the houses didnt change much between each other, other than mMüstakil. So I decided to convert anything anything thats not null or Müstakil to bilinmiyor

In [255]:
avg_prices = df.groupby("Tapu_Durumu")["Fiyat"].mean()
print("Average House Price for Each Tapu_Durumu:")
print(avg_prices)

Average House Price for Each Tapu_Durumu:
Tapu_Durumu
Arsa Tapulu        3.713374e+06
Bilinmiyor         6.460287e+06
Hisseli Tapu       3.478537e+06
Kat Mülkiyeti      4.835997e+06
Kat İrtifakı       3.278187e+06
Müstakil Tapulu    9.460861e+06
Name: Fiyat, dtype: float64



# change the values to either bilinmiyor or müstakil tapulu we will convert null later
keep_values = ["Bilinmiyor", "Müstakil Tapulu"]

# Convert every other non-null value to "Bilinmiyor"
df["Tapu_Durumu"] = df["Tapu_Durumu"].apply(
    lambda x: x if (pd.isna(x) or x in keep_values) else "Bilinmiyor"
)
output_file = "home_price_cleaned.csv"
df.to_csv(output_file, index=False)
print(df["Tapu_Durumu"].unique())


as for the null values the houses that have <= than 2 floors >= 4 rooms and that are over 4.1 million are classified as Müstakil while the rest are bilinmiyor


# Load data
df = pd.read_csv("home_price_cleaned.csv")

print("Initial rows:", len(df))

# ---------------------------------------------
# 1) FIX ODA_SAYISI (ROOM COUNT) VALUES
# ---------------------------------------------

print("\nFixing Oda_Sayısı...")

# If values are floats like 2.5 → 3, use ceiling
df["Oda_Sayısı"] = np.ceil(df["Oda_Sayısı"])

# Convert to integer type safely
df["Oda_Sayısı"] = df["Oda_Sayısı"].astype("Int64")

print("Unique room counts after fix:", df["Oda_Sayısı"].unique())

# ---------------------------------------------
# 2) FILL Tapu_Durumu ONLY WHERE NaN
# ---------------------------------------------

print("\nApplying Tapu_Durumu rule...")
changed_to_mustakil = 0
changed_to_bilinmiyor = 0

def fill_tapu(row):
    global changed_to_mustakil, changed_to_bilinmiyor
    
    if pd.isna(row["Tapu_Durumu"]):
        if (row["Fiyat"] > 4_100_000) and (row["Oda_Sayısı"] >= 4) and (row["Binanın_Kat_Sayısı"] <= 2):
            changed_to_mustakil += 1
            return "Müstakil Tapulu"
        else:
            changed_to_bilinmiyor += 1
            return "Bilinmiyor"
    else:
        return row["Tapu_Durumu"]

df["Tapu_Durumu"] = df.apply(fill_tapu, axis=1)

# ---------------------------------------------
# 3) PRINT SUMMARY
# ---------------------------------------------

print("\n== Tapu_Durumu Filling Summary ==")
print("Converted to Müstakil Tapulu:", changed_to_mustakil)
print("Converted to Bilinmiyor:", changed_to_bilinmiyor)
print(df["Tapu_Durumu"].value_counts(dropna=False))

# ---------------------------------------------
# 4) SAVE RESULT TO A NEW FILE
# ---------------------------------------------

output_file = "home_price_cleaned.csv"
df.to_csv(output_file, index=False)

print(f"\nSaved cleaned file as: {output_file}")


Only Bulunduğu_Kat remain

In [256]:
# Verify final null values other than bulunduğu_kat
print(df.isnull().sum())
rows_with_nulls = df.isnull().any(axis=1).sum()
print(f"Number of rows with null values: {rows_with_nulls}")

Net_Metrekare             0
Brüt_Metrekare            0
Oda_Sayısı                0
Bulunduğu_Kat          2272
Eşya_Durumu               0
Binanın_Yaşı              0
Isıtma_Tipi               0
Fiyat                     0
Şehir                     0
Binanın_Kat_Sayısı        0
Kullanım_Durumu           0
Yatırıma_Uygunluk         0
Takas                     0
Tapu_Durumu           10472
Banyo_Sayısı              0
dtype: int64
Number of rows with null values: 11587


In [257]:
 
df["Tapu_Durumu"] = df["Tapu_Durumu"].fillna("Bilinmiyor")
output_file = "home_price_cleaned.csv"
df.to_csv(output_file, index=False)

print(f"\nSaved cleaned file as: {output_file}")
# Show rows that contain any null values
null_rows = df[df.isnull().any(axis=1)]
print(df.isnull().sum())
print(f"Rows with null values: {len(null_rows)}")
print(df['Yatırıma_Uygunluk'].unique())




Saved cleaned file as: home_price_cleaned.csv
Net_Metrekare            0
Brüt_Metrekare           0
Oda_Sayısı               0
Bulunduğu_Kat         2272
Eşya_Durumu              0
Binanın_Yaşı             0
Isıtma_Tipi              0
Fiyat                    0
Şehir                    0
Binanın_Kat_Sayısı       0
Kullanım_Durumu          0
Yatırıma_Uygunluk        0
Takas                    0
Tapu_Durumu              0
Banyo_Sayısı             0
dtype: int64
Rows with null values: 2272
['Bilinmiyor' 'Uygun']


In [258]:
 # fill null values in Bulunduğu_Kat with "Bilinmiyor"
df["Bulunduğu_Kat"] = df["Bulunduğu_Kat"].fillna("Bilinmiyor")
output_file = "home_price_cleaned.csv"
df.to_csv(output_file, index=False)

print(f"\nSaved cleaned file as: {output_file}")
# Show rows that contain any null values
null_rows = df[df.isnull().any(axis=1)]
print(df.isnull().sum())
print(f"Rows with null values: {len(null_rows)}")


Saved cleaned file as: home_price_cleaned.csv
Net_Metrekare         0
Brüt_Metrekare        0
Oda_Sayısı            0
Bulunduğu_Kat         0
Eşya_Durumu           0
Binanın_Yaşı          0
Isıtma_Tipi           0
Fiyat                 0
Şehir                 0
Binanın_Kat_Sayısı    0
Kullanım_Durumu       0
Yatırıma_Uygunluk     0
Takas                 0
Tapu_Durumu           0
Banyo_Sayısı          0
dtype: int64
Rows with null values: 0


Now lets Encode the data turning all object type variables to numeric

In [259]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20131 entries, 0 to 20130
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Net_Metrekare       20131 non-null  int64  
 1   Brüt_Metrekare      20131 non-null  float64
 2   Oda_Sayısı          20131 non-null  float64
 3   Bulunduğu_Kat       20131 non-null  object 
 4   Eşya_Durumu         20131 non-null  object 
 5   Binanın_Yaşı        20131 non-null  object 
 6   Isıtma_Tipi         20131 non-null  object 
 7   Fiyat               20131 non-null  float64
 8   Şehir               20131 non-null  object 
 9   Binanın_Kat_Sayısı  20131 non-null  int64  
 10  Kullanım_Durumu     20131 non-null  object 
 11  Yatırıma_Uygunluk   20131 non-null  object 
 12  Takas               20131 non-null  object 
 13  Tapu_Durumu         20131 non-null  object 
 14  Banyo_Sayısı        20131 non-null  float64
dtypes: float64(4), int64(2), object(9)
memory usage: 2.3+

In [ ]:

import re
# Replace specified categories with "Düz Giriş (Zemin)"
ground_level_values = [
    'Yüksek Giriş',
    'Müstakil',
    'Bahçe Katı',
    'Bahçe Dublex',
    'Villa Tipi'
]

df['Bulunduğu_Kat'] = df['Bulunduğu_Kat'].replace(ground_level_values, 'Düz Giriş (Zemin)')

def convert_underground_to_bodrum(value):
    if pd.isna(value):
        return value
    
    text = str(value)

    # Detect underground levels like "-1", "-2", "Kot 1 (-1).Kat", etc.
    if re.search(r'-\d+', text):  
        return "Bodrum Kat"
    
    return text

df["Bulunduğu_Kat"] = df["Bulunduğu_Kat"].apply(convert_underground_to_bodrum)

def normalize_floor(value):
    if pd.isna(value):
        return value
    
    text = str(value)

    # 1️⃣ Convert underground floors (negative numbers) → Bodrum Kat
    if re.search(r'-\d+', text):
        return "Bodrum Kat"

    # 2️⃣ Convert garden/entrance variants → Düz Giriş (Zemin)
    if text in ["Yüksek Giriş", "Müstakil", "Bahçe Katı", "Bahçe Dublex", "Villa Tipi"]:
        return "Düz Giriş (Zemin)"

    # 3️⃣ Convert Çatı Dubleks → Çatı Katı
    if "Çatı Dubleks" in text:
        return "Çatı Katı"

    return text

df["Bulunduğu_Kat"] = df["Bulunduğu_Kat"].apply(normalize_floor)
output_file = "home_price_cleaned.csv"
df.to_csv(output_file, index=False)
print(df["Bulunduğu_Kat"].unique())




['4.Kat' '3.Kat' '6.Kat' 'Düz Giriş (Zemin)' '12.Kat' '2.Kat' 'Bilinmiyor'
 '8.Kat' '5.Kat' '14.Kat' '16.Kat' '1.Kat' '17.Kat' '9.Kat' '7.Kat'
 '11.Kat' '10.Kat' '15.Kat' '13.Kat' 'Bodrum Kat' '18.Kat' 'Çatı Katı'
 '21.Kat' '26.Kat' '40+.Kat' '19.Kat' '30.Kat' '22.Kat']


In [261]:
# Display column information
df.info()
print(df['Bulunduğu_Kat'].unique())
output_file = "home_price_cleaned.csv"
df.to_csv(output_file, index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20131 entries, 0 to 20130
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Net_Metrekare       20131 non-null  int64  
 1   Brüt_Metrekare      20131 non-null  float64
 2   Oda_Sayısı          20131 non-null  float64
 3   Bulunduğu_Kat       20131 non-null  object 
 4   Eşya_Durumu         20131 non-null  object 
 5   Binanın_Yaşı        20131 non-null  object 
 6   Isıtma_Tipi         20131 non-null  object 
 7   Fiyat               20131 non-null  float64
 8   Şehir               20131 non-null  object 
 9   Binanın_Kat_Sayısı  20131 non-null  int64  
 10  Kullanım_Durumu     20131 non-null  object 
 11  Yatırıma_Uygunluk   20131 non-null  object 
 12  Takas               20131 non-null  object 
 13  Tapu_Durumu         20131 non-null  object 
 14  Banyo_Sayısı        20131 non-null  float64
dtypes: float64(4), int64(2), object(9)
memory usage: 2.3+

In [262]:
import pandas as pd

# Load your CSV
df = pd.read_csv("home_price_cleaned.csv")

# Column names (adjust if needed)
CITY_COL = "Şehir"
PRICE_COL = "Fiyat"

# 1. Average price per city (descending)
city_avg = df.groupby(CITY_COL)[PRICE_COL].mean().sort_values(ascending=False)

# 2. Split cities into 3 groups
cities = city_avg.index.tolist()
num_groups = 3
group_size = len(cities) // num_groups

city_to_group = {}

for i, city in enumerate(cities):
    if i < group_size:
        city_to_group[city] = "group1_expensive"
    elif i < group_size * 2:
        city_to_group[city] = "group2_medium"
    else:
        city_to_group[city] = "group3_cheap"

# 3. Replace the Şehir column values with grouped labels
df[CITY_COL] = df[CITY_COL].map(city_to_group)

# 4. Show result
df.info()
print(df['Şehir'].unique())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20131 entries, 0 to 20130
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Net_Metrekare       20131 non-null  int64  
 1   Brüt_Metrekare      20131 non-null  float64
 2   Oda_Sayısı          20131 non-null  float64
 3   Bulunduğu_Kat       20131 non-null  object 
 4   Eşya_Durumu         20131 non-null  object 
 5   Binanın_Yaşı        20131 non-null  object 
 6   Isıtma_Tipi         20131 non-null  object 
 7   Fiyat               20131 non-null  float64
 8   Şehir               20131 non-null  object 
 9   Binanın_Kat_Sayısı  20131 non-null  int64  
 10  Kullanım_Durumu     20131 non-null  object 
 11  Yatırıma_Uygunluk   20131 non-null  object 
 12  Takas               20131 non-null  object 
 13  Tapu_Durumu         20131 non-null  object 
 14  Banyo_Sayısı        20131 non-null  float64
dtypes: float64(4), int64(2), object(9)
memory usage: 2.3+

import pandas as pd

# Read the CSV file
df = pd.read_csv('home_price_cleaned.csv')  

# Remove columns
df = df.drop(columns=['Bulunduğu_Kat'], errors='ignore')

# Save back to the same file
df.to_csv('home_price_cleaned.csv', index=False)

print("Columns removed successfully and file overwritten!")
df.info()

import pandas as pd
from sklearn.preprocessing import LabelEncoder

# 1. Load your dataset
input_file = 'home_price_cleaned.csv'
output_file = 'home_price_cleaned_OHE.csv'

df = pd.read_csv(input_file)

# 2. Define the categorical columns to encode
categorical_cols = [
    'Bulunduğu_Kat',
    'Eşya_Durumu',
    'Binanın_Yaşı', 
    'Isıtma_Tipi',
    'Şehir',
    'Kullanım_Durumu',
    'Yatırıma_Uygunluk',
    'Takas',
    'Tapu_Durumu'
]

# 3. Apply Label Encoding
label_encoders = {}

for col in categorical_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col].astype(str))
    label_encoders[col] = le

# 4. Save the encoded dataset
df.to_csv(output_file, index=False)

print(f"Label Encoding complete. Data saved to {output_file}")
print(f"Original shape: {df.shape}")
print(f"New shape: {df.shape}")

# 5. Reload and show info
df = pd.read_csv(output_file)
df.info()


One Hot Encoding

In [263]:
import pandas as pd

# 1. Load your dataset
# Replace 'your_dataset.csv' with your actual file name
input_file = 'home_price_cleaned.csv'
output_file = 'home_price_cleaned_OHE.csv' 

df = pd.read_csv(input_file)

# 2. Define the categorical columns to encode
# These are selected based on the 'object' type in your info list
categorical_cols = [
    'Bulunduğu_Kat',
    'Eşya_Durumu',
    'Binanın_Yaşı', 
    'Isıtma_Tipi',
    'Şehir',
    'Kullanım_Durumu',
    'Yatırıma_Uygunluk',
    'Takas',
    'Tapu_Durumu'
]

# 3. Apply One-Hot Encoding
# dtype=int ensures the output is 0 and 1 instead of True and False
df_encoded = pd.get_dummies(df, columns=categorical_cols, dtype=int)

# 4. Save the changes to a file
# index=False ensures we don't save the row numbers as a separate column
df_encoded.to_csv(output_file, index=False)

print(f"Encoding complete. Data saved to {output_file}")
print(f"Original shape: {df.shape}")
print(f"New shape: {df_encoded.shape}")
df = pd.read_csv('home_price_cleaned_OHE.csv')
df.info()

Encoding complete. Data saved to home_price_cleaned_OHE.csv
Original shape: (20131, 15)
New shape: (20131, 124)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20131 entries, 0 to 20130
Columns: 124 entries, Net_Metrekare to Tapu_Durumu_Müstakil Tapulu
dtypes: float64(4), int64(120)
memory usage: 19.0 MB


A viusalization and calculation of variables in our dataset

In [264]:
df = pd.read_csv('home_price_cleaned_OHE.csv')
null_rows = df[df.isnull().any(axis=1)]
print(df.isnull().sum())
print(f"Rows with null values: {len(null_rows)}")
df.info()
df.shape

Net_Metrekare                  0
Brüt_Metrekare                 0
Oda_Sayısı                     0
Fiyat                          0
Binanın_Kat_Sayısı             0
                              ..
Tapu_Durumu_Bilinmiyor         0
Tapu_Durumu_Hisseli Tapu       0
Tapu_Durumu_Kat Mülkiyeti      0
Tapu_Durumu_Kat İrtifakı       0
Tapu_Durumu_Müstakil Tapulu    0
Length: 124, dtype: int64
Rows with null values: 0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20131 entries, 0 to 20130
Columns: 124 entries, Net_Metrekare to Tapu_Durumu_Müstakil Tapulu
dtypes: float64(4), int64(120)
memory usage: 19.0 MB


(20131, 124)

In [265]:
import pandas as pd
import numpy as np

def review_and_optionally_delete(df, condition, description):


    outliers = df[condition]

    if outliers.empty:
        print(f"\nNo outliers found for: {description}")
        return df
    
    print(f"\n==============================")
    print(f"OUTLIERS FOUND: {description}")
    print(f"Count: {len(outliers)}")
    print(outliers.head(10))  # show first 10 outliers
    print("==============================")

    choice = input("Delete these rows? (y/n): ").strip().lower()
    
    if choice == "y":
        print("Deleting rows...\n")
        df = df[~condition]
    else:
        print("Keeping all rows.\n")

    return df

# Make a working copy
clean_df = pd.read_csv("home_price_cleaned_OHE.csv")


condition = ~clean_df["Net_Metrekare"].between(10, 2000)
clean_df = review_and_optionally_delete(clean_df, condition,
                                        "Unrealistic Net_Metrekare (<10 or >2000)")


condition = ~clean_df["Brüt_Metrekare"].between(20, 4000)
clean_df = review_and_optionally_delete(clean_df, condition,
                                        "Unrealistic Brüt_Metrekare (<20 or >4000)")


condition = clean_df["Net_Metrekare"] > clean_df["Brüt_Metrekare"]
clean_df = review_and_optionally_delete(clean_df, condition,
                                        "Net_Metrekare greater than Brüt_Metrekare")


condition = ~clean_df["Oda_Sayısı"].between(0.5, 15)
clean_df = review_and_optionally_delete(clean_df, condition,
                                        "Unrealistic Oda_Sayısı (not between 0.5 and 15)")

condition = ~clean_df["Banyo_Sayısı"].between(0, 10)
clean_df = review_and_optionally_delete(clean_df, condition,
                                        "Unrealistic Banyo_Sayısı (not between 0 and 10)")

condition = clean_df["Banyo_Sayısı"] > clean_df["Oda_Sayısı"] + 2
clean_df = review_and_optionally_delete(clean_df, condition,
                                        "Banyo_Sayısı > Oda_Sayısı + 2")

# Negative or too small price
condition = clean_df["Fiyat"] <= 300000
clean_df = review_and_optionally_delete(clean_df, condition,
                                        "Fiyat ≤ 300000 TL (incorrect low price)")

condition = clean_df["Fiyat"] >= 500_000_000
clean_df = review_and_optionally_delete(clean_df, condition,
                                        "Fiyat ≥ 500M TL (unrealistic high price)")


condition = ~clean_df["Binanın_Kat_Sayısı"].between(1, 200)
clean_df = review_and_optionally_delete(clean_df, condition,
                                        "Unrealistic Binanın_Kat_Sayısı (not between 1 and 200)")

output_filename = "home_price_encoded_outlier.csv"
clean_df.to_csv(output_filename, index=False, encoding='utf-8-sig')
print(f"\n✓ Cleaned data saved to: {output_filename}")
print(f"✓ File contains {len(clean_df)} rows and {len(clean_df.columns)} columns")


print("\n" + "="*50)
print("CLEANING COMPLETE - SUMMARY")
print("="*50)
print(f"Original dataset rows: {len(df)}")
print(f"Cleaned dataset rows:  {len(clean_df)}")
print(f"Total rows removed:    {len(df) - len(clean_df)}")
print(f"Removal percentage:    {(len(df) - len(clean_df)) / len(df) * 100:.2f}%")
print("="*50)

# Optional: Show first few rows of cleaned data
if input("\nShow first 5 rows of cleaned data? (y/n): ").lower() == "y":
    print("\nFirst 5 rows of cleaned data:")
    print(clean_df.head())


OUTLIERS FOUND: Unrealistic Net_Metrekare (<10 or >2000)
Count: 47
      Net_Metrekare  Brüt_Metrekare  Oda_Sayısı        Fiyat  \
187            3698          3700.0         4.0    3350000.0   
1039              1             2.0         5.0    2980000.0   
1054              2             3.0         3.0    2380000.0   
1898           2661          2665.0         3.0    2550000.0   
2003          12500         25000.0         5.5   13250000.0   
2016          11308         11310.0         7.0    1250000.0   
2089              5            40.0         2.0     950000.0   
2353           4499          4500.0         8.0   14500000.0   
3417          16500         17000.0         7.0  235000000.0   
3577              2            45.0         2.0    6500000.0   

      Binanın_Kat_Sayısı  Banyo_Sayısı  Bulunduğu_Kat_1.Kat  \
187                    1           1.0                    0   
1039                  13           2.0                    0   
1054                  15           2.0